In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.linkedin.com/jobs/view/4214589033")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
pg_data = loader.load().pop().page_content
pg_data

'\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nMercedes-Benz Mobility AG hiring Working Student in Data, Analytics and BI starting June 2025 in Stuttgart, Baden-Württemberg, Germany | LinkedIn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\nLinkedIn and 3rd parties use essential and non-essential cookies to provide, secure, analyze and improve our Services, and to show you relevant ads (including professional and job ads) on and off LinkedIn. Learn more in our Cookie Policy.Select Accept to consent or Reject to decline non-essential cookies for this use. You can update your choices at any time in your settings.\n\n\n\n\n                Accept\n                \n\n                Reject\n                \n\n\n \n\n\n\n\n\n      Skip to main content\n    \n\n\n\nLinkedIn\n\n\n\n\n\n\n\n              Working Student in Data, Analytics and BI starting June 2025 in Stuttgart\n \nExpand search\n\n\n\n\n\n\nThis button displays the currently selected search type. Wh

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"
)

In [10]:
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {pg_data}

    ### INSTRUCTUION
    The scrapped text is from the job page of LinkedIn. Your task is to extract the following details from the job posting and return them in a JSON format with the keys:
    `job_title`, `role`, `description`, `skills`, `experience`, `tasks`.
    Only return the valid JSON format.

    ### VALID JSON (NO PREAMBLE)
    """
)

# creating a langchain chain
chain_extract = prompt_extract | llm # getting a prompt and passing it to the llm

res = chain_extract.invoke(input={'pg_data': pg_data})

print(res.content)
print()
print(type(res.content))

```json
{
  "job_title": "Working Student in Data, Analytics and BI",
  "role": "Internship",
  "description": "Mercedes-Benz Mobility AG is hiring a Working Student in Data, Analytics and BI starting June 2025 in Stuttgart, Baden-Württemberg, Germany",
  "skills": [
    "MS Office suite (Word, Excel, PowerPoint)",
    "JIRA",
    "Strong communication skills",
    "Ability to work collaboratively in a team environment",
    "Interest in exploring and understanding the world of data"
  ],
  "experience": "Currently enrolled as a student in a relevant field",
  "tasks": [
    "Understand Mercedes-Benz internal and external requirements for secure and compliant data transfer in the cloud",
    "Develop an understanding of our customers' data solutions and requirements",
    "Coordinate and communicate effectively with both the team and customers",
    "Adhere to rule processes, including the thorough review of documents and deliverables",
    "Document relevant information and maintain a

In [11]:
# using the output parser to convert the str json content into a json type
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_obj = json_parser.parse(res.content)

json_obj

{'job_title': 'Working Student in Data, Analytics and BI',
 'role': 'Internship',
 'description': 'Mercedes-Benz Mobility AG is hiring a Working Student in Data, Analytics and BI starting June 2025 in Stuttgart, Baden-Württemberg, Germany',
 'skills': ['MS Office suite (Word, Excel, PowerPoint)',
  'JIRA',
  'Strong communication skills',
  'Ability to work collaboratively in a team environment',
  'Interest in exploring and understanding the world of data'],
 'experience': 'Currently enrolled as a student in a relevant field',
 'tasks': ['Understand Mercedes-Benz internal and external requirements for secure and compliant data transfer in the cloud',
  "Develop an understanding of our customers' data solutions and requirements",
  'Coordinate and communicate effectively with both the team and customers',
  'Adhere to rule processes, including the thorough review of documents and deliverables',
  'Document relevant information and maintain accurate records of compliance activities',
  

In [12]:
type(json_obj)

dict

In [61]:
# Getting the skills data to store in chromadb
import pandas as pd

df = pd.read_csv('skills.csv')
df

,TechStack,Link1,Link2
0,React,https://github.com/anniechakraborty/anniechak...,https://github.com/anniechakraborty/textutils
1,Angular,https://github.com/anniechakraborty/simpleKan...,https://github.com/anniechakraborty/frontend-...
2,Flask,https://github.com/anniechakraborty/simpleKan...,https://github.com/anniechakraborty/PersonalW...
3,MongoDB,https://github.com/anniechakraborty/simpleKan...,https://github.com/anniechakraborty/BookQuest
4,Python,https://github.com/anniechakraborty/simpleKan...,https://github.com/anniechakraborty/BookQuest
5,GenAI,https://github.com/anniechakraborty/GenAIProj...,https://github.com/anniechakraborty/cold-emai...
6,ReactNative,https://github.com/anniechakraborty/TextHelp,https://github.com/anniechakraborty/my-app
7,"HTML, CSS, TailwindCSS",https://github.com/anniechakraborty/manage-la...,https://github.com/anniechakraborty/social-li...
8,Machine Learning,https://github.com/anniechakraborty/Classific...,https://github.com/anniechakraborty/Regressio...
9,"XML, Xquery",https://github.com/anniechakraborty/BookQuest,https://github.com/anniechakraborty/BookQuest


In [62]:
# Preparing the chromaDB as a persistent client to store the data on a disk so that we can access the same data later on without having to insert it again
# using just client stores the data in-memory

import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='Portfolio')
print(df.columns.tolist())
if not collection.count():
    for _, row in df.iterrows():
        primary_link = str(row['Link1']) if pd.notna(row['Link1']) else ''
        secondary_link = str(row['Link2']) if pd.notna(row['Link2']) else ''
        tech_stack = str(row['TechStack']) if pd.notna(row['TechStack']) else ''
        collection.add(
            documents=[tech_stack],
            metadatas={
                'primary_links': primary_link,
                'secondary_links': secondary_link
            },
            ids=[str(uuid.uuid4())]
        )

['TechStack', 'Link1', 'Link2']


In [63]:
print(collection.peek())
# collection.delete(
#     ids=['710bb5f3-b95f-4497-a816-611f3c569343', '32224923-7d2e-4075-a147-7d1fcaff0718', '920fff09-2502-409a-8a16-b80d841075e4', 'f19a6f6c-4090-4e58-9140-6ac2fa41edc7', 'a8f9b1b9-c0b6-45cc-a23e-985abb7eb2b8', '3950196c-92d0-4017-8262-d5a196a77638', '96d687ee-47c3-4892-800c-e6805ed01d75', 'd7664246-0a58-4db2-87bd-1ce50549e984', 'e20d5376-087e-493c-a645-f0fd73ecfc32', '6a4e31e7-a1a6-4c8d-8fbe-fe61b0589b62']
# )
collection.count()

{'ids': ['dcdf6d24-64a5-4e68-b4e4-c87eb826bf3a', 'd777530e-c320-47b0-9c3e-12de272bb028', '404fbf68-8863-4dc6-a85b-dc2cd2e78b97', '9ee1d38e-0f51-4e5a-80ab-1e9f959de076', '47dd32d0-4929-4224-928d-6ae68eefb042', '47198e47-cb6d-44d2-9f31-37955247981c', 'd3de57b6-1a3a-4b13-a77d-4979a0ddc261', '515e45e0-d307-4b6f-8527-628a93169b61', 'd2a867d8-a956-4804-83cf-44ec97a3537d', '8c2815a8-c17b-47ec-a969-001fcfeec19f'], 'embeddings': array([[-0.08726904,  0.00863323,  0.03240667, ..., -0.01087549,
         0.03119643, -0.01890796],
       [-0.09454282,  0.03366246, -0.10063555, ...,  0.01545668,
         0.00646383, -0.05681656],
       [-0.08751205,  0.00945656, -0.06109304, ...,  0.07134798,
         0.04969556, -0.02960439],
       ...,
       [ 0.00566728,  0.05578964,  0.01165499, ...,  0.04982461,
         0.00703615,  0.11334594],
       [-0.02439189,  0.00324442,  0.05426768, ...,  0.03536279,
        -0.02179913, -0.05384536],
       [ 0.03419754,  0.06116725,  0.01045411, ...,  0.03658027,

10

In [ ]:
# getting relevant portfolio links for some given skills

links = collection.query(
    query_texts=['Python', 'XML'],
    n_results=1
).get('metadatas')

links

[[{'primary_links': ' https://github.com/anniechakraborty/simpleKanbanBackend',
   'secondary_links': ' https://github.com/anniechakraborty/BookQuest'},
  {'primary_links': ' https://github.com/anniechakraborty/simpleKanbanBackend',
   'secondary_links': ' https://github.com/anniechakraborty/PersonalWebsiteBackend'}],
 [{'secondary_links': ' https://github.com/anniechakraborty/BookQuest',
   'primary_links': ' https://github.com/anniechakraborty/BookQuest'},
  {'primary_links': ' https://github.com/anniechakraborty/simpleKanbanBackend',
   'secondary_links': ' https://github.com/anniechakraborty/BookQuest'}]]